# Introduction

Our project objective was first to simulate combat using the rules and frameworks of the popular tabletop role-playing game (TTRPG), Dungeons and Dragons (D&D), and then to use those simulated combat scenarios to determine the effectiveness over time of being able to guess an opponent's relavant combat statistics.  

In a TTRPG combat scenario, one character or set of characters (called an adventuring party, or simply "party," for short) engages another character or party in battle. In D&D, the players know the relevant statistics for their own characters, and sometimes for their entire party, while the Dungeon Master, or DM, (the chief storyteller of the game, who provides the scenarios and context for the player characters to engage with) often obscures the relevant statistics of the opposing character(s). Usually, the players are entirely unaware of what statistics, skills, and special abilities an enemy combatant posesses until they are exposed to its moveset, attack, and defense abilities in real-time during a battle. Knowing this, we wondered: with a basic understanding of Bayesian probability, how easy or quick would it be for a player party to be able to correctly guess an enemy combatant's relevant statistics in battle?

In order to understand what goes into that process, let's first talk a bit more about the way combat in a TTRPG, namely D&D, is run.

# Dungeons and Dragons Combat Mechanics

Combat, in the world of Dungeons and Dragons, is a surprisingly civilized affair: opposing parties operate on a turn-based system, determined initially by which character is quickest to act (through an "Initiative Roll"), followed by the next, down the line and looping back around to the same order in the next round of combat. During one's turn, a character can act in a few different ways. They can make an Attack, which may be either Melee, Ranged, Magical, or in some cases, using a Special Ability. They can Move, a number of feet based on their Speed statistic. They can also perform Bonus Actions or act upon any other Special Abilities they have. 

An Attack hits an opponent if the Attack Roll (1 roll of a 20-sided die, known as a d20, plus the attacker's relevant weapon modifier and Proficiency Bonus, if applicable) is greater than or equal to the defender's Armor Class, or AC (the character's Dexterity modifier, plus 10, plus any bonus from worn armor). Damage is determined by a roll of a die, often either a d4, a d6, or a d8, depending on the weapon used, and the damge is subtracted by the character's total amount of Hit Points, or HP. When a character's HP is 0 or less, the character is unconscious and cannot take further action in battle. Combat ends when one full party is decommissioned from battle, meaning all characters are unconscious or dead, or if one party flees or surrenders in some other manner.

In our model, we have made several simplifications and allowances for ease of following combat proceedings. Since a character's special abilities can become hyper-specialized, further abilities are gained by increasing character level, and the amount of magical combat spells in D&D is exorbitant, we decided to stick to Melee weapons combat, giving each character a weapon that utilizes either their Strength statistic or their Dexterity statistic. Additionally, we decided to ignore speed in an effort for this to not evolve into a project involving game theory, strategy interviews, or any models for random motion. This is D&D combat in its purest, simplest form: a bunch of characters standing around and hitting each other. What could be better?

In [6]:
"""
Module containing classes that represent characters
"""
import random
from typing import *


class Character:
    def __init__(self, name: str, hp: int, dex: int, stren: int, prof: int, weapon_die: int, finesse: bool):
        """
        Creates a new character with the given stats
        :param name: a string, this character's name (and UID)
        :param hp: an int, the maximum HP
        :param dex: an int, the Dexterity score
        :param stren: an int, the Strength score
        :param prof: an int, the character's proficiency bonus
        :param weapon_die: an int, the size of the die that is rolled to determine a weapon's damage
        :param finesse: a boolean, True if the weapon is finesse
        """
        self.name = name
        self.max_hp = hp
        self.hp = hp
        self.dex = dex
        self.stren = stren
        self.prof = prof
        self.weapon_die = weapon_die
        self.finesse = finesse

    @property
    def dex_mod(self) -> int:
        """
        :return: an int, this character's dexterity modifier
        """
        return (self.dex - 10) // 2

    @property
    def str_mod(self) -> int:
        """
        :return: an int, this character's strength modifier
        """
        return (self.stren - 10) // 2

    @property
    def weapon_bonus(self) -> int:
        """
        :return: an int, the character's best bonus
        """
        return max(self.dex_mod, self.str_mod) if self.finesse else self.str_mod

    @property
    def ac(self) -> int:
        """
        :return: an int, this character's armor class
        """
        return 10 + self.dex_mod

    def roll_initiative(self) -> Tuple[int, int]:
        """
        :return: a tuple of ints, this character's initiative followed by their dex score (for tiebreaks)
        """
        return random.randint(1, 20) + self.dex_mod, self.dex

    def roll_to_hit(self) -> int:
        """
        :return: an int, the value this character has rolled to hit
        """
        return random.randint(1, 20) + self.weapon_bonus + self.prof

    def roll_damage(self) -> int:
        """
        :return: an int, the amount of damage this character does
        """
        return random.randint(1, self.weapon_die) + self.weapon_bonus

    def __hash__(self):
        return hash(self.name)

After defining the Character class with relevant statistics, we moved on to simulating the battle. We defined 3 characters, 2 in the player party and 1 in the DM party, using our Character class, then wrote a function that would make them roll Initiative, and in turn order, beat each other up until both members of one party hit 0 HP. Then, we printed messages letting us know which damage was dealt in each turn and a message letting us know which party had won.

In [7]:
def simulate_battle(verbose: bool = False, plot: bool = False):
    party1 = [Character('Manster Wipower', 90, 10, 10, 0, 8, False),
              Character('Abayes Satano\'brien', 150, 14, 10, 0, 10, False)]
    party2 = [Character('Blelduth Chestsplitter', 240, 14, 10, 0, 8, False)]
    mod_est = {c.name: ModifierEstimator() for c in party2}
    ac_est = {c.name: ACEstimator() for c in party2}
    all_characters = {c.name: c for c in party1 + party2}
    initiative = {n: c.roll_initiative() for n, c in all_characters.items()}
    initiative_order = sorted(all_characters.keys(), key=lambda i: initiative[i], reverse=True)

    ac_full_range = ac_est[party2[0].name].xvals
    acrange = [max(ac_full_range) - min(ac_full_range) + 1]
    mod_full_range = ac_est[party2[0].name].xvals
    modrange = [max(mod_full_range) - min(mod_full_range) + 1]
    p1hp = [sum(c.hp for c in party1)]
    p2hp = [sum(c.hp for c in party2)]

    if verbose:
        print(f'Initiative order is {initiative_order} (scores are {initiative})')
    while True:
        for name in initiative_order:
            character = all_characters[name]
            is_party1 = any(c.name == name for c in party1)
            if character.hp <= 0:
                continue
            other_party = party2 if is_party1 else party1
            if all(c.hp <= 0 for c in other_party):
                break
            opponent = random.choice(other_party)
            while opponent.hp <= 0:
                opponent = random.choice(other_party)

            to_hit = character.roll_to_hit()
            hit = to_hit >= opponent.ac
            if hit:
                damage = character.roll_damage()
                opponent.hp -= damage
                if verbose:
                    print(f'{name} hit {opponent.name} for {damage} damage (they have {opponent.hp} HP left)')
                    if opponent.hp <= 0:
                        print(f'{opponent.name} is DEAD')

            if is_party1:
                ac_est[opponent.name].update((to_hit, hit))
                poss = ac_est[opponent.name].xvals[ac_est[opponent.name].current_estimate > 0]
                acrange.append(max(poss) - min(poss) + 1)
            else:
                mod_est[name].update(to_hit)
                poss = mod_est[name].xvals[mod_est[name].current_estimate > 0]
                modrange.append(max(poss) - min(poss) + 1)
        p1hp.append(sum(c.hp for c in party1))
        p2hp.append(sum(c.hp for c in party2))
        # if verbose:
        #     display_est_plots(mod_est, ac_est)
        if all(c.hp <= 0 for c in party1):
            if verbose:
                print('Freak the Mighty wins!')
            break
        if all(c.hp <= 0 for c in party2):
            if verbose:
                print('Hell Raisers win!')
            break
        if acrange[-1] == 1 and modrange[-1] == 1:
            acest = ac_est[party2[0].name]
            modest = mod_est[party2[0].name]
            assert acest.xvals[acest.current_estimate > 0][0] == party2[0].ac
            assert modest.xvals[modest.current_estimate > 0][0] == party2[0].weapon_bonus + party2[0].prof
            break


After we had created a working simulation of a simplified combat scenario, it was time to introduce our means of estimation.